## 1. 必要なライブラリをimport!

In [ ]:
import lightgbm as lgbm
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 2. ドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. パスを通して読み込む

In [ ]:
cd drive/My Drive/GCI2020/Competition3/Model

In [ ]:
input_dir = '../input/'
genres = pd.read_csv(input_dir + 'genres.csv')
goods = pd.read_csv(input_dir + 'goods.csv')
stores = pd.read_csv(input_dir + 'stores.csv')
train = pd.read_csv(input_dir + 'train.csv')
test = pd.read_csv(input_dir + 'test.csv')
sample_submission = pd.read_csv(input_dir + 'sample_submission.csv')

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
train_df = train.copy()
test_df = test.copy()

## 4. 前処理
### 4.1 データ11ヶ月分の取り込みとカラムに月を追加、正解ラベルの作成
    testデータにあって、月ごとのtrainデータにないgoods_idとstore_idの組み合わせを0で埋める

In [ ]:
def separate_and_groupe_by_month(train_df, test_df):
    """月別に分けて月カラムの追加と正解ラベル作成関数(11ヶ月分)"""

    sep_month_train_df_list = []
    for i in range(-11, 0):
        sep_month_train_df_list.append(train_df[train_df['num_month']==i])

    sep_train_df_list = []
    padding_df = test_df[test_df['goods_id']==-1]
    for i in range(11):
        sep_train_df_list.append(pd.DataFrame(sep_month_train_df_list[i].groupby(['store_id', 'goods_id'])['units_sold_day'].sum()).reset_index())
        sep_train_df_list[i] = sep_train_df_list[i].rename(columns={'units_sold_day': 'units_sold_month'})
    
    have_to_concat_df = test_df.drop('index', axis=1)
    for i in range(11):
        sep_train_df_list[i] = pd.concat([sep_train_df_list[i], have_to_concat_df]).drop_duplicates(subset=['store_id', 'goods_id'])
        sep_train_df_list[i]['units_sold_month'] = sep_train_df_list[i]['units_sold_month'].fillna(0)
        sep_train_df_list[i]['month'] = 11 - i

    return sep_train_df_list

In [ ]:
sep_train_df_list = separate_and_groupe_by_month(train_df, test_df)

### 4.2 月別に売上個数の平均を取り、monthのエンコーディング用辞書を作成
    考えとして、ロシアの冬は恐らく外に出たくないので、買い込む傾向があるのかなぁ……と。
    下記のように月ごとにカラムで残っているので、dictでmap関数を使ってエンコーディングする
    平成26年11月〜平成27年9月: 11〜1
    平成27年10月: 0

In [ ]:
def preprocess_units_sold_month_encoding(df_list):
    """monthへの月別個数平均Encodingの辞書作成関数"""

    mean_units_sold_per_month_list = []
    for df in df_list:
        month_mean = df['units_sold_month'].mean()
        mean_units_sold_per_month_list.append(month_mean)

    print("過去11ヶ月間の月別の個数平均↓", "\n", mean_units_sold_per_month_list)

    per_month_dict = {}
    for i in range(11):
        per_month_dict[11-i] = mean_units_sold_per_month_list[i]
    per_month_dict[0] = per_month_dict[1]

    return per_month_dict

In [ ]:
per_month_dict = preprocess_units_sold_month_encoding(sep_train_df_list)

平成27年10月(testデータ)は平成27年9月と同じにする

In [ ]:
test_df['month'] = 0

### 4.3 goods_genre_idのエンコーディング用辞書を作成
    商品のカテゴリ(goods_genre_id)ごとにgoods_idのPriceを平均とることで作成

In [ ]:
train_df_11 = train_df[train_df['num_month'] > -12]

In [ ]:
def add_goods_info(df):
    """train/testとGoodsのテーブルをくっつける関数"""

    goods_df = pd.merge(df, goods[['goods_id', 'goods_genre_id']], on='goods_id')
    
    return goods_df

In [ ]:
def pre_encode_goods_genre_id_dict(df):
    """グッズのカテゴリ変数を商品の平均PriceでEncoding"""

    train_df_goods = add_goods_info(df)

    goods_genre_id_dict = {}
    for index, value in train_df_goods.groupby('goods_genre_id')['price'].mean().items():
        goods_genre_id_dict[index] = value

    return goods_genre_id_dict

In [ ]:
goods_genre_id_dict = pre_encode_goods_genre_id_dict(train_df_11)

### 4.4 店舗別で売上を計算して、store_idのエンコーディング用辞書を作成

In [ ]:
def pre_encode_store_id_dict(train_df):
    """店舗別で月別売上でエンコーディング(Uriage Encoding)"""
    
    train_df['uriage'] = train_df['price'] * train_df['units_sold_day']
    store_uriage_df = train_df.groupby(['store_id'])['uriage'].sum().reset_index()
    store_uriage_df['uriage'] /= np.mean(store_uriage_df['uriage'].values)

    store_id_dict = {}
    for store_id, uriage in store_uriage_df.values:
        store_id_dict[int(store_id)] = uriage

    return store_id_dict

In [ ]:
store_id_dict = pre_encode_store_id_dict(train_df_11)

In [ ]:
all_train_df = pd.concat([df for df in sep_train_df_list], axis=0)

In [ ]:
all_df = pd.concat([all_train_df, test_df], axis=0)

In [ ]:
all_df_goods = add_goods_info(all_df)

### 4.5 shift特徴量の作成
    nori0724さんのDiscussionで上がっていたベースラインNotebookからとってきました笑

In [ ]:
shift_monthes = [1,2,3,4,5,6]
for shift_month in tqdm(shift_monthes):
    all_df_goods[f"target_shift_m{shift_month}"] = all_df_goods.groupby(['store_id', 'goods_id'])["units_sold_month"].transform(lambda x: x.shift(shift_month))

In [ ]:
all_df_goods.to_csv('../input/all_df_goods_shift_6_monthes.csv', index=False)

In [ ]:
# all_df_goods = pd.read_csv('../input/all_df_goods_shift_6_monthes.csv')

### 4.5 map関数で辞書を使ってエンコーディング

In [ ]:
def modify_map(df, store_id_dict, goods_genre_id_dict, per_month_dict):
    """map使う関数"""

    featured_all_df = df.copy()
    featured_all_df['store_id'] = df['store_id'].map(store_id_dict)
    featured_all_df['goods_genre_id'] = df['goods_genre_id'].map(goods_genre_id_dict)
    featured_all_df['month'] = df['month'].map(per_month_dict)

    return featured_all_df

In [ ]:
featured_all_df = modify_map(all_df_goods, store_id_dict, goods_genre_id_dict, per_month_dict)

## 5. trainデータとtestデータで分割して、必要なカラムを取る

In [ ]:
df_train = featured_all_df[featured_all_df['index'].isnull()].drop('index', axis=1).reset_index(drop=True)
df_test = featured_all_df[~featured_all_df['index'].isnull()].sort_values('index').reset_index(drop=True)

In [ ]:
train_X = df_train.drop(columns=['units_sold_month'])
train_y = df_train['units_sold_month']
test_X = df_test.drop(columns=['index', 'units_sold_month'])

In [ ]:
display(train_X.head(), train_y.head(), test_X.head())

In [ ]:
per_month_dict

In [ ]:
CV_month_list = [1]

## 6. Optunaによるパラメーターチューニング
    CV: 5.879939908947618

In [ ]:
lgbm_best_param = {'reg_lambda': 0.06875461198761648, 'reg_alpha': 2.2293651627955215, 'colsample_bytree': 0.7208495854217002, 'subsample': 0.781401201566485, 'max_depth': 8, 'min_child_weight': 3.109569481392587}

## 7. 学習
    CVをできるようにしているが、精度は良くならなかったので、平成27年9月のみでValidationを行っている

In [ ]:
def fit_lgbm(train_X, train_y, CV_month_list, per_month_dict, params: dict=None, verbose=100):
    
    oof_preds = []

    if params is None:
        params = {}

    models = []

    score_list = []

    for i, num in enumerate(CV_month_list):
        per_month = per_month_dict[num]
        x_train = train_X[train_X['month']!=per_month]
        y_train = train_y[train_X['month']!=per_month]
        x_valid = train_X[train_X['month']==per_month]
        y_valid = train_y[train_X['month']==per_month]

        clf = lgbm.LGBMRegressor(**params, random_state=71, n_estimators=10000, learning_rate=0.05)
        clf.fit(x_train, y_train,
                eval_set=[(x_valid, y_valid)], 
                early_stopping_rounds=100, 
                eval_metric='rmse',
                verbose=verbose)
          
        oof_preds.append(clf.predict(x_valid, num_iteration=clf.best_iteration_))

        models.append(clf)
        score_list.append(np.sqrt(mean_squared_error(y_valid, oof_preds[i])))
        print('Fold %2d RMSE : %.6f' % (i + 1, score_list[i]))
    
    score = np.mean(score_list)
    print('Full RMSE score %.6f' % score) 
    return oof_preds, models, score_list

In [ ]:
oof, models, scores = fit_lgbm(train_X, train_y, CV_month_list=CV_month_list, per_month_dict=per_month_dict, params=lgbm_best_param)

## 8. カラムの重要度を可視化

In [ ]:
lgbm.plot_importance(models[0], importance_type="gain", max_num_features=40, figsize=(12, 12))

## 9. 予測

In [ ]:
pred = np.array([model.predict(test_X.values) for model in models])

In [ ]:
print(len(pred), len(pred[0]))

In [ ]:
submission = sample_submission.copy()
submission['units_sold_month'] = pred[0]

In [ ]:
submission.info()

In [ ]:
submission

In [ ]:
submission.to_csv('../submissions/3rd_place_solution.csv', index=False)